# DistilBERT (CPU)

## Setup

### Packages Setup

#### Install Packages

In [33]:
%pip install datasets
%pip install evaluate
%pip install hf_xet
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install tensorflow
%pip install tf-keras
%pip install transformers
%pip install transformers[torch]
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Packages

In [ ]:
from datasets import Dataset, Value
import evaluate
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, balanced_accuracy_score, brier_score_loss


















from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding  
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer 
from torch.utils.data import DataLoader

### Data Setup

#### Read Data

In [3]:
train_cleaned = pd.read_json("../data/processed/IMDB_reviews_train_cleaned.json")
test = pd.read_json("../data/processed/IMDB_reviews_test.json")

train_cleaned.head()
test.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
391376,24 October 2006,tt0424136,ur0023796,False,Most films do best if you know next to nothing...,9,Hard Candy breaks minds as hard candy breaks t...
573647,2 September 2001,tt0139239,ur1235973,False,Go has not gotten even half of the praise it d...,9,One of the most under appreciated films in his...
426616,3 March 2011,tt0480249,ur24994931,False,Personally I really enjoyed this movie from th...,7,Why Do People Hate This Movie?
493566,11 March 2004,tt0103874,ur0395246,False,"As far as videos go, this is one of the few th...",6,Aye shoood tayhke thee trahyne tew Byoodapest
174694,11 May 2013,tt1931533,ur17825945,True,While trying a little too hard to be Adaptatio...,4,Unlucky Number Seven


In [22]:
train_cleaned['is_spoiler'] = train_cleaned['is_spoiler'].astype('int64') 
train_cleaned = train_cleaned.rename(columns={'is_spoiler': 'labels', 'review_text': 'text'})

test['is_spoiler'] = test['is_spoiler'].astype('int64') 
test = train_cleaned.rename(columns={'is_spoiler': 'labels', 'review_text': 'text'})

In [23]:
train_cleaned.loc[0]

review_date                                        10 February 2006
movie_id                                                  tt0111161
user_id                                                   ur1898687
labels                                                            1
text              oscar year shawshank redemption written direct...
rating                                                           10
review_summary        A classic piece of unforgettable film-making.
Name: 0, dtype: object

In [24]:
train_cleaned.dtypes

review_date       object
movie_id          object
user_id           object
labels             int64
text              object
rating             int64
review_summary    object
dtype: object

In [26]:
train_dataset = Dataset.from_pandas(train_cleaned[['text', 'labels']])
train_dataset = train_dataset.cast_column('labels', Value('int64'))
test_dataset = Dataset.from_pandas(test[['text', 'labels']])
test_dataset = test_dataset.cast_column('labels', Value('int64'))
print(train_dataset)
print(test_dataset)

Casting the dataset: 100%|██████████| 459130/459130 [00:00<00:00, 999123.59 examples/s] 

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 459130
})
Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 459130
})


### Model Setup

#### DistilBERT

In [ ]:
model_name = "distilbert-base-uncased"
num_labels = 2  # For spoiler/non-spoiler classification
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels, problem_type="single_label_classification")
distilbert_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Tokenizer Setup

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

## Processing

### Data Processing

#### Tokenize Data

Tokenize the data and rename is_spoiler to labels so the transformer model can recognize as y value.

In [30]:
tokenized_train_eval = train_dataset.map(tokenize, batched=True, )
tokenized_train_eval = tokenized_train_eval.remove_columns(["text"])
tokenized_train_eval.set_format(type='torch')

tokenized_test = test_dataset.map(tokenize, batched=True)
tokenized_test = tokenized_test.remove_columns(["text"])
tokenized_test.set_format(type='torch')

Map: 100%|██████████| 459130/459130 [03:10<00:00, 2412.01 examples/s]


In [32]:
first = tokenized_train_eval[0]
print(type(first['labels']), first['labels']) # with set_format('torch'), this is a torch.Tensor

<class 'torch.Tensor'> tensor(0)


In [35]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)
loader = DataLoader(tokenized_train_eval, batch_size=16, collate_fn=collator)
batch = next(iter(loader))
print(batch['labels'].dtype, batch['labels'].shape) # should be torch.int64 (Long) and shape [batch]

torch.int64 torch.Size([16])


#### Split Train and Eval Data

In [36]:
split_datasets = tokenized_train_eval.train_test_split(test_size=0.2, seed=42)

tokenized_train = split_datasets['train']
tokenized_eval = split_datasets['test']

## Modeling

### DistilBERT

#### Model Initialization

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy", # or F1-score, precision, recall
)

trainer = Trainer(
    model=distilbert_model,
    args=training_args,
    train_dataset=tokenized_train, # Your tokenized training data
    eval_dataset=tokenized_eval,   # Your tokenized evaluation data
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics_function, # Optional: define a function to compute metrics
)

C:\Users\sandr\AppData\Local\Temp\ipykernel_33548\1271776542.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#### Train Model

In [42]:
trainer.train()

c:\Users\sandr\Documents\NUS\NUS_Courses\Y3S1\CS3244\Project\imdb-spoilers\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Evaluate Model

### DistilBERT

#### Predict Test Data

In [ ]:
predictions = trainer.predict(tokenized_test)
# Process predictions to determine spoiler/non-spoiler

#### Evaluate Predictions

In [ ]:
print("Test Metrics:", predictions.metrics)

logits = predictions.predictions
labels = predictions.label_ids

predicted_class_ids = np.argmax(logits, axis=-1)

metric = evaluate.load("f1")
f1_score = metric.compute(predictions=predicted_class_ids, references=labels, average="weighted")
print(f"F1 Score on test set: {f1_score}")

In [ ]:
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1)

# Probabilities for the positive class (index 1)
e_x = np.exp(logits - logits.max(axis=1, keepdims=True))
probs = e_x / e_x.sum(axis=1, keepdims=True)
prob_pos = probs[:, 1]

# Evaluate metrics
accuracy = evaluate.load("accuracy").compute(predictions=preds, references=labels)["accuracy"]
precision = evaluate.load("precision").compute(predictions=preds, references=labels, average="binary")["precision"]
recall = evaluate.load("recall").compute(predictions=preds, references=labels, average="binary")["recall"]
f1_binary = evaluate.load("f1").compute(predictions=preds, references=labels, average="binary")["f1"]
f1_macro = evaluate.load("f1").compute(predictions=preds, references=labels, average="macro")["f1"]
f1_weighted = evaluate.load("f1").compute(predictions=preds, references=labels, average="weighted")["f1"]
roc_auc = evaluate.load("roc_auc").compute(references=labels, prediction_scores=prob_pos)["roc_auc"]
avg_precision = evaluate.load("average_precision").compute(references=labels, prediction_scores=prob_pos)["average_precision"]

Extra (sklearn)
mcc = matthews_corrcoef(labels, preds)
balanced_acc = balanced_accuracy_score(labels, preds)
brier = brier_score_loss(labels, prob_pos)
cm = confusion_matrix(labels, preds, labels=[0, 1][0, 1])
report = classification_report(labels, preds, target_names=["non_spoiler", "spoiler"], digits=4)

print("Test Metrics:")
print(f"- accuracy: {accuracy:.4f}")
print(f"- precision (binary): {precision:.4f}")
print(f"- recall (binary): {recall:.4f}")
print(f"- f1 (binary): {f1_binary:.4f}")
print(f"- f1 (macro): {f1_macro:.4f}")
print(f"- f1 (weighted): {f1_weighted:.4f}")
print(f"- ROC-AUC: {roc_auc:.4f}")
print(f"- PR-AUC (average_precision): {avg_precision:.4f}")
print(f"- MCC: {mcc:.4f}")
print(f"- balanced_accuracy: {balanced_acc:.4f}")
print(f"- Brier score: {brier:.4f}")
print("Confusion matrix [[TN, FP], [FN, TP]]:")
print(cm)
print("Classification report:")
print(report)

Optional: compute_metrics for Trainer (so these are logged during eval)

# Load metrics once, then define compute_metrics:
acc_m = evaluate.load("accuracy")
prec_m = evaluate.load("precision")
rec_m = evaluate.load("recall")
f1_m = evaluate.load("f1")
roc_m = evaluate.load("roc_auc")
ap_m = evaluate.load("average_precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    e_x = np.exp(logits - logits.max(axis=1, keepdims=True))
    prob_pos = (e_x / e_x.sum(axis=1, keepdims=True))[:, 1]
    return {
        "accuracy": acc_m.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_m.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": rec_m.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1": f1_m.compute(predictions=preds, references=labels, average="binary")["f1"],
        "f1_macro": f1_m.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": f1_m.compute(predictions=preds, references=labels, average="weighted")["f1"],
        "roc_auc": roc_m.compute(references=labels, prediction_scores=prob_pos)["roc_auc"],
        "average_precision": ap_m.compute(references=labels, prediction_scores=prob_pos)["average_precision"],
        "mcc": matthews_corrcoef(labels, preds),
        "balanced_accuracy": balanced_accuracy_score(labels, preds),
        "brier": brier_score_loss(labels, prob_pos),
    }

# Then pass compute_metrics=compute_metrics to Trainer.

Notes:

For ROC-AUC/PR-AUC you must use the positive-class probability (prob_pos).
If you prefer a different positive class, adjust which column you take from probs.
Your current F1 (weighted) line is fine; you can keep it alongside the others.